In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/the-liar-dataset/data.csv


In [3]:
import pandas as pd 
import numpy as np
from tqdm.auto import tqdm
import torch 
import torch.nn as nn
import torch.optim as optim 
from sklearn.model_selection import train_test_split
from transformers import DistilBertTokenizer , DistilBertModel
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from transformers import BertModel, BertTokenizer
import torch.nn as nn 
from torch.utils.data  import Dataset
from torch.utils.data import Dataset, DataLoader
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, accuracy_score
import joblib


In [4]:
data = pd.read_csv("/kaggle/input/the-liar-dataset/data.csv")

In [5]:
data.head()

,title,text,subject,date,label
0,BREAKING: GOP Chairman Grassley Has Had Enoug...,"Donald Trump s White House is in chaos, and th...",News,"July 21, 2017",0
1,Failed GOP Candidates Remembered In Hilarious...,Now that Donald Trump is the presumptive GOP n...,News,"May 7, 2016",0
2,Mike Pence’s New DC Neighbors Are HILARIOUSLY...,Mike Pence is a huge homophobe. He supports ex...,News,"December 3, 2016",0
3,California AG pledges to defend birth control ...,SAN FRANCISCO (Reuters) - California Attorney ...,politicsNews,"October 6, 2017",1
4,AZ RANCHERS Living On US-Mexico Border Destroy...,Twisted reasoning is all that comes from Pelos...,politics,"Apr 25, 2017",0


In [6]:
class SubjectDataset(Dataset):
    def __init__(self , df):
        self.df=df
        self.maxlen=256
        self.tokenizer=BertTokenizer.from_pretrained("bert-base-uncased")
    def __len__(self):
        return len(self.df)
    def __getitem__(self , index):
        sample_title= str(self.df['title'].iloc[index])
        sample_content= str(self.df['text'].iloc[index])
        sample = sample_title + " " + sample_content 
        encodings = self.tokenizer.encode_plus(
        sample,
        add_special_tokens=True,
        max_length=self.maxlen,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt' 
        )
        label = torch.tensor(self.df['label'].iloc[index], dtype=torch.long)
        return {
            'input_ids': encodings['input_ids'].flatten(),  
            'attention_mask': encodings['attention_mask'].flatten(),
            'labels': label
        } 

In [7]:
def get_bert_embeddings(dataset, bert_model, device):
    embeddings = [] # Will store BERT embeddings for each text
    labels = [] ## for labels
    bert_model = bert_model.to(device)
    bert_model.eval()
    
    loader = DataLoader(dataset, batch_size=32, shuffle=False)
    # Disable gradient calculations since we're only doing inference not classiftion 
    with torch.no_grad():
        for batch in tqdm(loader, desc="Getting BERT embeddings"):
            input_ids = batch['input_ids'].to(device) ## batch to dvice [batch = 16 in this case _size, 256]
            attention_mask = batch['attention_mask'].to(device) ## same here for attention_mask
            
            outputs = bert_model(input_ids=input_ids, attention_mask=attention_mask)
            # Get [CLS] token embeddings
            # Extract [CLS] token embeddings
            # outputs.last_hidden_state shape: [batch_size, 256, 768]
            # [:, 0, :] selects the first token ([CLS]) of each sequence
            batch_embeddings = outputs.last_hidden_state[:, 0, :].cpu().numpy()
            embeddings.extend(batch_embeddings)
            labels.extend(batch['labels'].cpu().numpy())
        # Convert lists to numpy arrays
    return np.array(embeddings), np.array(labels)

In [8]:
# def main():
#     df = pd.read_csv("/kaggle/input/the-liar-dataset/data.csv") ## for kaggek
#     ## df = pd.read_csv("data.csv") ## for lical
#     device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#     print(f'im using ',device)
#     df_train , df_test = train_test_split(df,train_size=0.8,random_state=42)    
#     df_train = SubjectDataset(df_train)
#     df_test = SubjectDataset(df_test) 
#     # train_loader = DataLoader(
#     #     df_train,
#     #     batch_size=32,
#     #     shuffle=True,
#     # )

#     # test_loader = DataLoader(
#     #     df_test,
#     #     batch_size=32,
#     #     shuffle=False,
#     # )
#     bert_model = BertModel.from_pretrained('bert-base-uncased')
#         # Get BERT embeddings
#     print("getting train enbaging bert")
#     X_train, y_train = get_bert_embeddings(df_train, bert_model, device)
#     print("getting test enbaging bert")
#     X_test, y_test = get_bert_embeddings(df_test, bert_model, device)
#      # Train Random Forest
#     print("Training Random Forest")
#     # param_grid = {
#     # 'bootstrap': [True, False],
#     # 'max_depth': [5, 10, 20, 30, 40, 50],
#     # 'max_features': ['auto', 'sqrt', 'log2'],
#     # 'min_samples_leaf': [1, 2, 4],
#     # 'min_samples_split': [2, 5, 10],
#     # 'n_estimators': [200, 400, 600, 800, 1000],
#     # 'criterion': ['gini', 'entropy']
#     #                 }
#     param_grid = {
#     'bootstrap': [True],  # Reduce options
#     'max_depth': [10, 20, 30],  # Fewer depth options
#     'max_features': ['auto'],  # Fewer feature selection methods
#     'min_samples_leaf': [1, 2],
#     'min_samples_split': [2, 5],
#     'n_estimators': [200, 400, 600],
#     'criterion': ['gini']
#     }

#     # Initialize base model
#     rf_base = RandomForestClassifier(random_state=42, n_jobs=-1)

#     # Initialize GridSearchCV
#     grid_search = GridSearchCV(
#         estimator=rf_base,
#         param_grid=param_grid,
#         cv=5,                  # 5-fold cross-validation
#         n_jobs=-1,            # Use all CPU cores
#         verbose=2,            # Detailed output
#         scoring='accuracy'    # Metric to optimize
#     )
#     print("Starting Grid Search...")
#     grid_search.fit(X_train, y_train)
#     # Print best parameters and score
#     print("\nBest parameters found:")
#     print(grid_search.best_params_)
#     print(f"\nBest cross-validation score: {grid_search.best_score_:.4f}")
#     # Evaluate on test set
#     best_rf = grid_search.best_estimator_
#     y_pred = best_rf.predict(X_test)
#     print("\nTest Set Performance:")
#     print(classification_report(y_test, y_pred))
#      # Save model (optional)
#     joblib.dump(best_rf, 'news_classifier_rf.joblib')
#     return bert_model , best_rf

In [9]:
def generate_bert_embeddings(df_path, model_name='bert-base-uncased'):
    # Chargement des données
    df = pd.read_csv(df_path)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    # Split des données
    df_train, df_test = train_test_split(df, train_size=0.8, random_state=42)
    
    # Chargement du modèle BERT
    bert_model = BertModel.from_pretrained(model_name)
    
    # Génération des embeddings
    print("Génération des embeddings pour l'ensemble d'entraînement")
    X_train, y_train = get_bert_embeddings(SubjectDataset(df_train), bert_model, device)
    
    print("Génération des embeddings pour l'ensemble de test")
    X_test, y_test = get_bert_embeddings(SubjectDataset(df_test), bert_model, device)
    
    # Sauvegarde des embeddings
    np.save('X_train_embeddings.npy', X_train)
    np.save('y_train_labels.npy', y_train)
    np.save('X_test_embeddings.npy', X_test)
    np.save('y_test_labels.npy', y_test)
    
    return X_train, y_train, X_test, y_test

In [10]:
def train_random_forest(X_train, y_train, X_test, y_test):
    # Espace de recherche très réduit
    param_grid = {
        'n_estimators': [200, 400],
        'max_depth': [10, 20],
        'min_samples_split': [2, 5],
        'criterion': ['gini']
    }
    
    # Modèle de base
    rf_base = RandomForestClassifier(random_state=42, n_jobs=-1)
    
    # Recherche par grille
    grid_search = GridSearchCV(
        estimator=rf_base,
        param_grid=param_grid,
        cv=3,  # Réduction du nombre de plis
        n_jobs=-1,
        verbose=1,
        scoring='accuracy'
    )
    
    # Entraînement
    grid_search.fit(X_train, y_train)
    
    # Meilleur modèle
    best_rf = grid_search.best_estimator_
    
    # Évaluation
    print("\nMeilleurs paramètres :")
    print(grid_search.best_params_)
    print(f"\nMeilleur score de validation croisée : {grid_search.best_score_:.4f}")
    
    # Prédictions sur l'ensemble de test
    y_pred = best_rf.predict(X_test)
    print("\nPerformance sur l'ensemble de test :")
    print(classification_report(y_test, y_pred))
    
    # Sauvegarde du modèle
    joblib.dump(best_rf, 'news_classifier_rf.joblib')
    
    return best_rf

In [11]:
def main():
    # Chemin vers votre fichier CSV
    df_path = "/kaggle/input/the-liar-dataset/data.csv"
    
    # Génération des embeddings
    X_train, y_train, X_test, y_test = generate_bert_embeddings(df_path)
    
    # Entraînement du Random Forest
    best_model = train_random_forest(X_train, y_train, X_test, y_test)
    
    return best_model

In [13]:
if __name__ == "__main__":
    bert_model, rf_model = main()

Génération des embeddings pour l'ensemble d'entraînement


Getting BERT embeddings:   0%|          | 0/1123 [00:00<?, ?it/s]

Génération des embeddings pour l'ensemble de test


Getting BERT embeddings:   0%|          | 0/281 [00:00<?, ?it/s]

Fitting 3 folds for each of 8 candidates, totalling 24 fits

Meilleurs paramètres :
{'criterion': 'gini', 'max_depth': 20, 'min_samples_split': 5, 'n_estimators': 400}

Meilleur score de validation croisée : 0.9558

Performance sur l'ensemble de test :
              precision    recall  f1-score   support

           0       0.96      0.96      0.96      4669
           1       0.96      0.96      0.96      4311

    accuracy                           0.96      8980
   macro avg       0.96      0.96      0.96      8980
weighted avg       0.96      0.96      0.96      8980



ValueError: too many values to unpack (expected 2)